In [1]:
import sys
import getpass
import re
from functools import partial
from tqdm import tqdm
import uuid

user = getpass.getuser()
sys.dont_write_bytecode = True

sys.path.insert(0, '/Users/{}/Box/DataViz Projects/Utility Code'.format(user))

from utils_io import *  # initial imports
reload_module('utils_io')  # reload after update to utils_io.py
from utils_io import *  # reload reflects updates

#geo imports
import geopandas as gp
import folium
from geopandas.tools import geocode
from geopy.geocoders import GoogleV3
from geopy.extra.rate_limiter import RateLimiter
from geopy import Point

In [2]:
google_api_key = G_CREDS['dataviz_team']

In [5]:
apr_id = 'briv-ikjp'

## Pull Geocoded Data and Run Through Geocoder w/ updates

In [119]:
apr = pull_df_from_socrata('briv-ikjp')

pulling data in 3 chunks of 14000 rows each
pulling chunk 0
pulling chunk 1
pulling chunk 2
took 6.4961 seconds


In [116]:
apr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30864 entries, 0 to 30863
Data columns (total 74 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   mtc_id                       30864 non-null  object 
 1   mtc_type                     30667 non-null  object 
 2   mtc_year                     30864 non-null  float64
 3   mtc_vlow_income_dr           254 non-null    float64
 4   mtc_vlow_income_ndr          371 non-null    float64
 5   mtc_vlow_tot                 30864 non-null  float64
 6   mtc_low_income_dr            388 non-null    float64
 7   mtc_low_income_ndr           821 non-null    float64
 8   mtc_low_tot                  30864 non-null  float64
 9   mtc_mod_income_dr            466 non-null    float64
 10  mtc_mod_income_ndr           3269 non-null   float64
 11  mtc_mod_tot                  30864 non-null  float64
 12  mtc_above_mod_income         25870 non-null  float64
 13  mtc_total_units 

In [125]:
columns = ['mtc_id',
           'mtc_address_full',
           'mtc_geocode_type']
apr_notgeo = apr[columns][apr['mtc_geocode_type'] == 'NOT_GEOCODED']

In [126]:
#Create a geo locator
g = GoogleV3(api_key=google_api_key)

In [127]:
#Use a rate limiter
bound_box = [Point(38.864245,-121.208156), Point(36.893329,-123.632497)]
geocode = RateLimiter(g.geocode, 
                      min_delay_seconds=.025, 
                      error_wait_seconds=1)

In [129]:
#Create a location column with geolocator and time the process 
tqdm.pandas()
bound_box = [Point(38.864245,-121.208156), Point(36.893329,-123.632497)]
apr_notgeo['location'] = (apr_notgeo['mtc_address_full']
                            .progress_apply(partial(geocode,
                                                    components={"country": "USA"},
                                                    timeout=1000,
                                                    bounds=bound_box)))


100%|██████████| 527/527 [05:39<00:00,  1.55it/s]


In [137]:
#Set geocode location type
apr_notgeo['mtc_geocode_type'] = apr_notgeo['location'].map(lambda loc: loc.raw['geometry']['location_type'] 
                                                                if loc else 'NOT_GEOCODED')

In [138]:
#Set latitude
apr_notgeo['mtc_lat'] = (apr_notgeo['location']
                                        .map(lambda loc: loc.latitude if loc else None))
#Set longitude 
apr_notgeo['mtc_long'] = (apr_notgeo['location']
                                        .map(lambda loc: loc.longitude if loc else None))

In [139]:
#Set geocode address 
apr_notgeo['mtc_geocode_address'] = (apr_notgeo['location']
                                      .map(lambda loc: loc.address if loc else None))

In [140]:
#Create geodataframe from lat long coords
apr_notgeo_geo = gp.GeoDataFrame(apr_notgeo,
                                  geometry=gp.points_from_xy(apr_notgeo['mtc_long'],apr_notgeo['mtc_lat']),
                                  crs="EPSG:4326")

In [147]:
#Set geometry field to null if no lat long
apr_notgeo_geo['geometry'] = np.where(apr_notgeo_geo['mtc_geocode_type']  == 'NOT_GEOCODED',
                                        np.nan,
                                        apr_notgeo_geo['geometry'])

In [148]:
#Set wkt column
apr_notgeo_geo['mtc_wkt'] = apr_notgeo_geo['geometry'].apply(lambda x: np.nan if x is None else x.wkt)

In [149]:
#Create checkbox column to indicate whether or not the project had been mapped
apr_notgeo_geo['mtc_mapped'] = np.where(apr_notgeo_geo['mtc_geocode_type'] == 'NOT_GEOCODED', False,True)

In [152]:
apr_geocode_updated_geo = apr_notgeo_geo[apr_notgeo_geo['mtc_mapped'] == True]

In [153]:
apr_geocode_updated_geo.shape

(68, 10)

In [154]:
final_columns = ['mtc_id',
                 'mtc_address_full',
                 'mtc_geocode_address',
                 'mtc_geocode_type',
                 'mtc_mapped',
                 'mtc_lat',
                 'mtc_long',
                 'mtc_wkt',
                 'mtc_mapped']

In [155]:
upsert_df_socrata(apr_geocode_updated_geo[final_columns],'briv-ikjp')

upserting data to Socrata in 1 chunks of 68 rows each


/Users/jcroff/Box/DataViz Projects/Utility Code/utils_analytics.py:1181: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  upsert_data_raw = df.to_dict(orient='records')


upserted chunk 0
data upserted to briv-ikjp
took 1.7823 seconds
